step function to query aws resources based on an input flag(resource)

In [ ]:
## StepFuntion Definition
{
  "Comment": "A simple AWS Step Functions state machine that automates a call center support session.",
  "StartAt": "ProcessTransaction",
  "States": {
    "ProcessTransaction": {
      "Type": "Choice",
      "Choices": [
        {
          "Variable": "$.resource",
          "StringEquals": "ec2",
          "Next": "listEC2"
        },
        {
          "Variable": "$.resource",
          "StringEquals": "ebs",
          "Next": "listEC2"
        },
        {
          "Variable": "$.resource",
          "StringEquals": "s3",
          "Next": "listS3"
        },
        {
          "Variable": "$.resource",
          "StringEquals": "lambda",
          "Next": "listLambda"
        }
      ]
    },
    "ListEC2": {
      "Type": "Task",
      "Resource": "arn:aws:lambda:us-east-2:636651395865:function:npListEC2",
      "End": true
    },
    "ListLambda": {
      "Type": "Task",
      "Resource": "arn:aws:lambda:us-east-2:636651395865:function:npListLambda",
      "End": true
    },
    "listS3": {
      "Type": "Task",
      "Resource": "arn:aws:lambda:us-east-2:636651395865:function:npListS3",
      "End": true
    }
  }
}

In [ ]:
##StepFunction IAMRole -need invoke permission thru an inline policy
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "lambda:InvokeFunction"
            ],
            "Resource": [
                "*"
            ]
        }
    ]
}

In [ ]:
##Lambda Function ListEC2
import json
import boto3
from collections import defultdict

def def_value():
    return []
def get_ec2s(myec2s):
    ec2data=defaultdict(def_value)
    response=myec2s.describe_instances()
    validresp=response.get('Reservations')
    if validresp:
        for each in validresp:
            instances=each.get('instances')
            #print(instances)
            if instances:
                for insta in instances:
                    #print(insta)
                    keys=['ImageId','State','PublicIP','DNS','PrivateIP']
                    #values=[insta.get(x) for x in keys]
                    values=[insta.get('ImageId'),insta.get('State')['Name'],insta.get('PublicIpAddress'),insta.get('PublicDnsName'),insta.get('PrivateIpAddress')]
                    atts=dict(zip(keys,values))
                    ec2data[insta.get('InstanceId')]=atts
    return ec2data
def get_volume(myec2s):
    regvols=defaultdict(def_value)
    result=myec2.describe_volume(Filters=[{'Name': 'status', 'Values':['in-use']}])
    for volume in result['Volumes']:
        attachments=volume['Attachments']
        instanceId=[]
        for attach in attachments:
            instanceId.append(attach['InstanceId'])
            #print(attach)
        voldict={volume['VolumeId']:{'size':volume['Size'],'instance': ','.join(instanceId)}}
        regvols[volume['AvailabilityZone']].append(voldict)
        print("A volume %s in %s "% (volume['VolumeId'], volume['AvailabilityZone']))
    return regvols
def lambda_handler(event,context):
    npresult={}
    ec2=boto3.client('ec2')
    regions = ec2.describe_region().get('Regions',[])
    for region in regions:
        print('Checking region %s '% region['RegionName'])
        reg=region['RegionName']
        ec2s=boto3.client('ec2',region_name=reg)
        result={}
        if event['resource'] == 'volume':
            result=get_volume(ec2s)
        elif event['resource'] == 'ec2':
            result=get_ec2s(ec2s)
        npresult[reg]=result
    return npresult
                    

In [ ]:
#Lambda Function ListEC2 IAMROLE
AWSLambdaBasicExecutionRole
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": "ec2:Describe",
            "Resource": "*"
        }
    ]
}
#lambda timeout >5 mins

In [ ]:
#Lambda Function ListS3
import boto3

def lambda_handler(event, context):

    npresult={}
    if event['resource'] =='s3':

        s3=boto3.resource('s3')
        for bucket in s3.buckets.all():
            total_size=0
            count=0
            for object in bucket.objects.all():
                count+=1
                total_size += object.size
            print(total_size)
            npresult[bucket.name]={'size':total_size,'count':count}
    return npresult


In [ ]:
#Lambda Function ListS3 IAMROLE
AWSLambdaBasicExecutionRole
{
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "s3:ListAllMyBuckets",
                "s3:GetObject"
            ],
            "Resource": "arn:aws:s3:::*"
        }
    ]
}
#lambda timeout >5 mins

In [ ]:
#Lambda Function ListLambda
import json
import boto3
from boto3.session import Session

def def_json():
    return []

def lambda_handler(event, context):
    npresult={}
    
    s=Session()
    lambs = s.get_available_regions('lambda')
    for reg in lambs:
        lambdas=boto3.client('lambda',region_name=reg)
        response=lambdas.list_functions()
        if response:
            print response
        npresult[reg]=response
    return npresult


In [ ]:
#Lambda Function ListLambda IAMROLE
AWSLambdaBasicExecutionRole
AWSLambda_FullAccess
#lambda timeout >5 mins